In [1]:
import os
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from utils import tools
from modules.vae_base_module import VAEBaseModule
from models.vae_simple import SimpleVAE
from datasets.novelty_mnist import NoveltyMNISTDataModule

%matplotlib inline
plt.style.use('seaborn')


In [ ]:
default_path = os.path.join(os.getcwd(), '../configs/vae/vae_simple_emnist.yaml')
config = tools.config_from_command_line(default_path)

In [ ]:
datamodule = NoveltyMNISTDataModule(**config['data-parameters'])
datamodule.setup('test')

model = SimpleVAE()

module = VAEBaseModule(model, **config['module-parameters'])

checkpoint = torch.load('../logs/VAESimple/version_0/checkpoints/epoch=35-step=12131.ckpt')
module.load_state_dict(checkpoint['state_dict'])

In [ ]:
test_novelty_scores = []
test_novelty_labels = []

module.model.eval()
with torch.no_grad():
    # enumerate dataloader
    # write test step for module
    # return novelty scores
